##Libraries

In [ ]:
import numpy as np
import os
import librosa
import librosa.display
from keras.models import Model,Sequential
from keras import optimizers
from keras.layers import Input,Conv1D,BatchNormalization,MaxPooling1D,LSTM,Dense,Activation,Layer,Conv2D,MaxPooling2D,Flatten,Reshape
from tensorflow.keras import regularizers
from keras.regularizers import l2
from keras.utils import to_categorical
import keras.backend as K
import argparse
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


##Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
classes = np.array(['W','L','E','A','F','T','S','N']) # 8 classes

Seconds_per_sample = 1 # The sample size in seconds
Padded_sample = True # Whether to take samples which are less than the required length
fs = 16000 # Sampling Frequency of Input data
seg_len = int((Seconds_per_sample)*16000) # signal split length (in samples) in time domain
seg_ov = int(seg_len*0.5) # 50% overlap

In [ ]:
# use raw time-domain speech signal as input to cnn for SER

def normalize(s):
# RMS normalization
	new_s = s/np.sqrt(np.sum(np.square((np.abs(s))))/len(s))
	return new_s

def countclasses(fnames):
	dict = {classes[0]:0,classes[1]:0,classes[2]:0,classes[3]:0,classes[4]:0,classes[5]:0,classes[6]:0,classes[7]:0}
	for name in fnames:
		if name[place] in classes:
			dict[name[place]]+=1
	return dict

def data1d(path):

	fnames = os.listdir(datapath)
	dict = countclasses(fnames)
	print('Total Data',dict)
	num_cl = len(classes)
	train_dict = {classes[0]:0,classes[1]:0,classes[2]:0,classes[3]:0,classes[4]:0,classes[5]:0,classes[6]:0,classes[7]:0}
	test_dict = {classes[0]:0,classes[1]:0,classes[2]:0,classes[3]:0,classes[4]:0,classes[5]:0,classes[6]:0,classes[7]:0}
	val_dict = {classes[0]:0,classes[1]:0,classes[2]:0,classes[3]:0,classes[4]:0,classes[5]:0,classes[6]:0,classes[7]:0}

	for i in range(num_cl):
		cname =  list(dict.keys())[i]
		cnum = dict[cname]
		t = round(0.8*cnum)
		test_dict[cname] = int(cnum - t)
		val_dict[cname] = int(round(0.2*t))
		train_dict[cname] = int(t - val_dict[cname])
		print('Class:',cname,'train:',train_dict[cname],'val:',val_dict[cname],'test:',test_dict[cname])
		
	x_train = []
	y_train = []
	x_test = []
	y_test = []
	x_val = []
	y_val = []

	count = {classes[0]:0,classes[1]:0,classes[2]:0,classes[3]:0,classes[4]:0,classes[5]:0,classes[6]:0,classes[7]:0}
	loop = 1
	for name in fnames:
		
		if name[place] in classes:
			sig,fs = librosa.load(datapath+'/'+name, sr=16000)
			if loop > 0:
					loop = -1
					print(name)
					print(sig.shape)
			
			# normalize signal
			data = normalize(sig)
			if(len(data) < seg_len):
				pad_len = int(seg_len - len(data))
				pad_rem = int(pad_len % 2)
				pad_len /= 2
				signal = np.pad(data,(int(pad_len), int(pad_len+pad_rem)),'constant',constant_values=0)
			elif(len(data) > seg_len):
				signal = []
				end = seg_len
				st = 0
				while(end < len(data)):
					signal.append(data[st:end])
					st = st + seg_ov
					end = st + seg_len
				signal = np.array(signal)
				if(end >= len(data)):
					num_zeros = int(end-len(data))
					if(num_zeros > 0):
						n1 = np.array(data[st:end])
						n2 = np.zeros([num_zeros])
						s = np.concatenate([n1,n2],0)
					else:
						s = np.array(data[int(st):int(end)])
				signal = np.vstack([signal,s])
			else:
				signal = data

			if(count[name[place]] < train_dict[name[place]]):
				if(signal.ndim>1):
					for i in range(signal.shape[0]):						
						x_train.append(signal[i])
						y_train.append(name[place])
				else:
					x_train.append(signal)
					y_train.append(name[place])
			else:
				if((count[name[place]]-train_dict[name[place]]) < val_dict[name[place]]):
					if(signal.ndim>1):
						for i in range(signal.shape[0]):							
							x_val.append(signal[i])
							y_val.append(name[place])
					else:
						x_val.append(signal)
						y_val.append(name[place])
				else:
					if(signal.ndim>1):
						for i in range(signal.shape[0]):
							x_test.append(signal[i])
							y_test.append(name[place])
					else:
						x_test.append(signal)
						y_test.append(name[place])
			count[name[place]]+=1
	return np.float32(x_train),y_train,np.float32(x_test),y_test,np.float32(x_val),y_val

def string2num(y):
	y1 = []
	for i in y:
		if(i == classes[0]):
			y1.append(0)
		elif(i == classes[1]):
			y1.append(1)
		elif(i == classes[2]):
			y1.append(2)
		elif(i == classes[3]):
			y1.append(3)
		elif(i == classes[4]):
			y1.append(4)
		elif(i == classes[5]):
			y1.append(5)
		elif(i == classes[6]):
			y1.append(6)
		else:
			y1.append(7)
	y1 = np.float32(np.array(y1))
	return y1

def load_data():
	x_tr,y_tr,x_t,y_t,x_v,y_v = data1d(datapath)
	y_tr = string2num(y_tr)
	y_t = string2num(y_t)
	y_v = string2num(y_v)
	return x_tr, y_tr, x_t, y_t, x_v, y_v

## Dataset Selection

In [ ]:
# !mkdir Emodb_dataset
# !unzip "/content/drive//My Drive/Emotion Detection/EmoDB.zip" -d Emodb_dataset;
datapath = 'Emodb_dataset/wav'
place = 5
xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val = load_data()

In [ ]:
# !mkdir Urdu_dataset
# !unzip "/content/drive/My Drive/Emotion Detection/Urdu Dataset.zip" -d Urdu_dataset;
datapath = 'Urdu_dataset/Urdu Dataset'
place = 0
xdu_tr,ydu_tr,xdu_t,ydu_t,xdu_val,ydu_val = load_data()

Total Data {'W': 100, 'L': 0, 'E': 0, 'A': 0, 'F': 100, 'T': 100, 'S': 0, 'N': 100}
Class: W train: 64 val: 16 test: 20
Class: L train: 0 val: 0 test: 0
Class: E train: 0 val: 0 test: 0
Class: A train: 0 val: 0 test: 0
Class: F train: 64 val: 16 test: 20
Class: T train: 64 val: 16 test: 20
Class: S train: 0 val: 0 test: 0
Class: N train: 64 val: 16 test: 20
T76.wav
(48057,)


In [ ]:
# !mkdir Emovo_dataset
# !unzip "/content/drive//My Drive/Emotion Detection/EMOVO_Dataset.zip" -d Emovo_dataset;
datapath = 'Emovo_dataset/EMOVO_Dataset'
place = 3
xdi_tr,ydi_tr,xdi_t,ydi_t,xdi_val,ydi_val = load_data()

Total Data {'W': 84, 'L': 84, 'E': 84, 'A': 84, 'F': 84, 'T': 84, 'S': 0, 'N': 84}
Class: W train: 54 val: 13 test: 17
Class: L train: 54 val: 13 test: 17
Class: E train: 54 val: 13 test: 17
Class: A train: 54 val: 13 test: 17
Class: F train: 54 val: 13 test: 17
Class: T train: 54 val: 13 test: 17
Class: S train: 0 val: 0 test: 0
Class: N train: 54 val: 13 test: 17
f1_A023.wav
(43691,)


In [ ]:
def appending(data = 'g'):
  if data == 'g':
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val
  elif data == 'u':
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdu_tr,ydu_tr,xdu_t,ydu_t,xdu_val,ydu_val
  elif data == 'i':
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdi_tr,ydi_tr,xdi_t,ydi_t,xdi_val,ydi_val
  elif data == 'gu':
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = np.append(xd_tr,xdu_tr,axis=0),np.append(yd_tr,ydu_tr,axis=0),np.append(xd_t,xdu_t,axis=0),np.append(yd_t,ydu_t,axis=0),np.append(xd_val,xdu_val,axis=0),np.append(yd_val,ydu_val,axis=0)
  elif data == 'gi':
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = np.append(xd_tr,xdi_tr,axis=0),np.append(yd_tr,ydi_tr,axis=0),np.append(xd_t,xdi_t,axis=0),np.append(yd_t,ydi_t,axis=0),np.append(xd_val,xdi_val,axis=0),np.append(yd_val,ydi_val,axis=0)
  elif data == 'iu':
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdi_tr,ydi_tr,xdi_t,ydi_t,xdi_val,ydi_val
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = np.append(xd_tr,xdu_tr,axis=0),np.append(yd_tr,ydu_tr,axis=0),np.append(xd_t,xdu_t,axis=0),np.append(yd_t,ydu_t,axis=0),np.append(xd_val,xdu_val,axis=0),np.append(yd_val,ydu_val,axis=0)
  else:
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = np.append(xd_tr,xdu_tr,axis=0),np.append(yd_tr,ydu_tr,axis=0),np.append(xd_t,xdu_t,axis=0),np.append(yd_t,ydu_t,axis=0),np.append(xd_val,xdu_val,axis=0),np.append(yd_val,ydu_val,axis=0)
    xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = np.append(xd_tr,xdi_tr,axis=0),np.append(yd_tr,ydi_tr,axis=0),np.append(xd_t,xdi_t,axis=0),np.append(yd_t,ydi_t,axis=0),np.append(xd_val,xdi_val,axis=0),np.append(yd_val,ydi_val,axis=0)
  return xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val

xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = appending(data = 'iu');
print(xd_tr.shape)
print(xd_val.shape)
print(xd_t.shape)
print(yd_tr.shape)
print(yd_val.shape)
print(yd_t.shape)


(3332, 16000)
(774, 16000)
(1069, 16000)
(3332,)
(774,)
(1069,)


## 1D CNN and LSTM

In [ ]:
# 1D cnn for SER

def emo1d(input_shape, num_classes,args):
	
	model = Sequential(name='Emo1D')
	
	# LFLB1
	model.add(Conv1D(filters = 64,kernel_size = (3),strides=1,padding='same',data_format='channels_last',input_shape=input_shape))	
	model.add(BatchNormalization())
	model.add(Activation('elu'))
	model.add(MaxPooling1D(pool_size = 4, strides = 4))

	#LFLB2
	model.add(Conv1D(filters=64, kernel_size = 3, strides=1,padding='same'))
	model.add(BatchNormalization())
	model.add(Activation('elu'))
	model.add(MaxPooling1D(pool_size = 4, strides = 4))

	#LFLB3
	model.add(Conv1D(filters=128, kernel_size = 3, strides=1,padding='same'))
	model.add(BatchNormalization())
	model.add(Activation('elu'))
	model.add(MaxPooling1D(pool_size = 4, strides = 4))

	#LFLB4
	model.add(Conv1D(filters=128, kernel_size = 3, strides=1,padding='same'))
	model.add(BatchNormalization())
	model.add(Activation('elu'))
	model.add(MaxPooling1D(pool_size = 4, strides = 4))

	#LSTM
	model.add(LSTM(units=args.num_fc)) 
		
	#FC
	model.add(Dense(units=100,activation='relu'))

	model.add(Dense(units=num_classes,activation='softmax'))

	#Model compilation	
	opt = optimizers.SGD(lr = args.learning_rate, decay=args.decay, momentum=args.momentum, nesterov=True)
	model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])
	# model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
	
	return model

def train(model,x_tr,y_tr,x_val,y_val,args):
	
	es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
	mc = ModelCheckpoint('best_model_1D_CNN.h5', monitor='val_categorical_accuracy', mode='max', verbose=1, save_best_only=True)
	history = model.fit(x_tr,y_tr,epochs=args.num_epochs,batch_size=args.batch_size,validation_data=(x_val,y_val),callbacks=[es, mc])
	return model

def test(model,x_t,y_t):

	saved_model = load_model('best_model_1D_CNN.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=20)
	print(score)
	return score

def loadData1d(x_tr,y_tr,x_t,y_t,x_val,y_val):

	x_tr = x_tr.reshape(-1,x_tr.shape[1],1)
	x_t = x_t.reshape(-1,x_t.shape[1],1)
	x_val = x_val.reshape(-1,x_val.shape[1],1)
	y_tr = to_categorical(y_tr)
	y_t = to_categorical(y_t)
	y_val = to_categorical(y_val)
	return x_tr,y_tr,x_t,y_t,x_val,y_val


In [ ]:
class arguments(object):        
		def __init__(self):
				self.num_fc = 256
				self.batch_size = 300
				self.num_epochs = 60 #best model will be saved before number of epochs reach this value
				self.learning_rate = 0.1
				self.decay = 1e-6
				self.momentum = 0.9

#load data	
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

print("Training Set Input Shape: ", x_tr.shape)
print("Validation Set Input Shape: ",x_val.shape)
print("Testing Set Input Shape: ",x_t.shape)
print("Training Set Output Shape: ",y_tr.shape)
print("Validation Set Output Shape: ",y_val.shape)
print("Testing Set Output Shape: ",y_t.shape)

args = arguments()

#define model
model = emo1d(input_shape=x_tr.shape[1:],num_classes=len(classes),args=args)
# model.summary()

#train model
model = train(model,x_tr,y_tr,x_val,y_val,args=args)

#test model
score = test(model,x_t,y_t)

Training Set Input Shape:  (3332, 16000, 1)
Validation Set Input Shape:  (774, 16000, 1)
Testing Set Input Shape:  (1069, 16000, 1)
Training Set Output Shape:  (3332, 8)
Validation Set Output Shape:  (774, 8)
Testing Set Output Shape:  (1069, 8)
Train on 3332 samples, validate on 774 samples
Epoch 1/60
3332/3332 [==============================] - 14s 4ms/step - loss: 1.8623 - categorical_accuracy: 0.2422 - val_loss: 1.9411 - val_categorical_accuracy: 0.2274

Epoch 00001: val_categorical_accuracy improved from -inf to 0.22739, saving model to best_model_1D_CNN.h5
Epoch 2/60
3332/3332 [==============================] - 13s 4ms/step - loss: 1.6942 - categorical_accuracy: 0.3223 - val_loss: 1.9032 - val_categorical_accuracy: 0.2080

Epoch 00002: val_categorical_accuracy did not improve from 0.22739
Epoch 3/60
3332/3332 [==============================] - 13s 4ms/step - loss: 1.5278 - categorical_accuracy: 0.3923 - val_loss: 2.5750 - val_categorical_accuracy: 0.2287

Epoch 00003: val_categor

In [ ]:
# !gsutil -q -m cp "best_model_1D_CNN.h5" "/content/drive/My Drive/Emotion Detection/Models/EMOVO_Urdu/MIX_8classes_1D_CNN_60%_1s.h5"

# Italian and Urdu Dataset Testing

## Italian and Urdu dataset Training and Testing

In [ ]:
# x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/EMOVO_Urdu/MIX_8classes_1D_CNN_62%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc2*b1 + acc3*c1)/(b1+c1))

Training
3332/3332 [==============================] - 5s 2ms/step
[0.039888284486239954, 0.9870948195457458]
Validation
774/774 [==============================] - 1s 2ms/step
[2.7020740102427876, 0.6240310072898865]
Testing
1069/1069 [==============================] - 2s 2ms/step
[2.455195940522193, 0.6239476203918457]
Overall accuracy = 0.623982640174311


## Italian and Urdu Dataset Training and German Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/EMOVO_Urdu/MIX_8classes_1D_CNN_62%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc1*a1 + acc2*b1 + acc3*c1)/(total))

Training
1733/1733 [==============================] - 3s 2ms/step
[4.71015308980507, 0.4223889112472534]
Validation
428/428 [==============================] - 1s 2ms/step
[5.960807965180584, 0.37616822123527527]
Testing
549/549 [==============================] - 1s 2ms/step
[5.5287887489860825, 0.3897996246814728]
Overall accuracy = 0.40848707595214634


## Italian and Urdu Dataset Training and Urdu Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdu_tr,ydu_tr,xdu_t,ydu_t,xdu_val,ydu_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/EMOVO_Urdu/MIX_8classes_1D_CNN_62%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',( acc2*b1 + acc3*c1)/(b1+c1))

Training
1150/1150 [==============================] - 2s 2ms/step
[0.12140697214914405, 0.9547826051712036]
Validation
270/270 [==============================] - 1s 2ms/step
[0.9498520029915704, 0.770370364189148]
Testing
375/375 [==============================] - 1s 2ms/step
[0.7206453883647919, 0.8026666641235352]
Overall accuracy = 0.7891472827556522


## Italian and Urdu Dataset Training and Italian Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdi_tr,ydi_tr,xdi_t,ydi_t,xdi_val,ydi_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/EMOVO_Urdu/MIX_8classes_1D_CNN_62%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',( acc2*b1 + acc3*c1)/(b1+c1))

Training
2182/2182 [==============================] - 3s 2ms/step
[0.052626368621283996, 0.9830430746078491]
Validation
504/504 [==============================] - 1s 2ms/step
[3.2808949379693892, 0.567460298538208]
Testing
694/694 [==============================] - 1s 2ms/step
[3.2405081578565262, 0.5201728940010071]
Overall accuracy = 0.5400667603505473


# German and Italian Dataset Testing

## German and Italian dataset Training and Testing

In [ ]:
# x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/Emodb_EMOVO/MIX_8classes_1D_CNN_55%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc2*b1 + acc3*c1)/(b1+c1))

Training
3915/3915 [==============================] - 6s 2ms/step
[0.13897559178058216, 0.9532567262649536]
Validation
932/932 [==============================] - 2s 2ms/step
[2.3402416644689863, 0.5568669438362122]
Testing
1243/1243 [==============================] - 2s 2ms/step
[2.2041123540576004, 0.5462590456008911]
Overall accuracy = 0.5508045909596586


## German and Italian Dataset Training and German Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/Emodb_EMOVO/MIX_8classes_1D_CNN_55%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',( acc2*b1 + acc3*c1)/(b1+c1))

Training
1733/1733 [==============================] - 3s 2ms/step
[0.1501575883540265, 0.9492210149765015]
Validation
428/428 [==============================] - 1s 2ms/step
[2.2489633404205893, 0.5887850522994995]
Testing
549/549 [==============================] - 1s 2ms/step
[2.297510679953737, 0.5591985583305359]
Overall accuracy = 0.5721596836311669


## German and Italian Dataset Training and Urdu Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdu_tr,ydu_tr,xdu_t,ydu_t,xdu_val,ydu_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/Emodb_EMOVO/MIX_8classes_1D_CNN_55%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',( acc2*b1 + acc3*c1)/(b1+c1))

Training
1150/1150 [==============================] - 2s 2ms/step
[8.920750327732252, 0.18956521153450012]
Validation
270/270 [==============================] - 1s 2ms/step
[9.087443616655138, 0.18518517911434174]
Testing
375/375 [==============================] - 1s 2ms/step
[8.803935585021973, 0.20266667008399963]
Overall accuracy = 0.1953488366548405


## German and Italian Dataset Training and Italian Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdi_tr,ydi_tr,xdi_t,ydi_t,xdi_val,ydi_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/Emodb_EMOVO/MIX_8classes_1D_CNN_55%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',( acc2*b1 + acc3*c1)/(b1+c1))

Training
2182/2182 [==============================] - 3s 2ms/step
[0.13009457523728815, 0.9564619660377502]
Validation
504/504 [==============================] - 1s 2ms/step
[2.417756023861113, 0.5297619104385376]
Testing
694/694 [==============================] - 1s 2ms/step
[2.130228182424386, 0.5360230803489685]
Overall accuracy = 0.5333889988507572


# German and Urdu Dataset Testing

## German and Urdu dataset Training and Testing

In [ ]:
# x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/Urdu_Emodb/MIX_8classes_1D_CNN_74%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc2*b1 + acc3*c1)/(b1+c1))

Training
2883/2883 [==============================] - 4s 2ms/step
[0.005213904846750363, 0.9989594221115112]
Validation
698/698 [==============================] - 1s 2ms/step
[1.8606714672892004, 0.7449856996536255]
Testing
924/924 [==============================] - 2s 2ms/step
[1.7028328722173518, 0.7564935088157654]
Overall accuracy = 0.7515413196695424


## German and Urdu Dataset Training and German Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/Urdu_Emodb/MIX_8classes_1D_CNN_74%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',( acc2*b1 + acc3*c1)/(b1+c1))

Training
1733/1733 [==============================] - 3s 2ms/step
[0.004837741220514497, 0.9988459348678589]
Validation
428/428 [==============================] - 1s 2ms/step
[2.2031035490125137, 0.6962617039680481]
Testing
549/549 [==============================] - 1s 2ms/step
[2.173030522351708, 0.6794171333312988]
Overall accuracy = 0.6867963311128021


## German and Urdu Dataset Training and Urdu Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdu_tr,ydu_tr,xdu_t,ydu_t,xdu_val,ydu_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/Urdu_Emodb/MIX_8classes_1D_CNN_74%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',( acc2*b1 + acc3*c1)/(b1+c1))

Training
1150/1150 [==============================] - 2s 2ms/step
[0.005780766351634393, 0.9991304278373718]
Validation
270/270 [==============================] - 1s 2ms/step
[1.3178536891937256, 0.8222222328186035]
Testing
375/375 [==============================] - 1s 2ms/step
[1.0144638389348983, 0.8693333268165588]
Overall accuracy = 0.8496124037476473


## German and Urdu Dataset Training and Emovo Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdi_tr,ydi_tr,xdi_t,ydi_t,xdi_val,ydi_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive/My Drive/Emotion Detection/Models/Urdu_Emodb/MIX_8classes_1D_CNN_74%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc1*a1 + acc2*b1 + acc3*c1)/(total))

Training
2182/2182 [==============================] - 3s 2ms/step
[11.107094072836675, 0.18010999262332916]
Validation
504/504 [==============================] - 1s 2ms/step
[10.607269832066127, 0.1865079402923584]
Testing
694/694 [==============================] - 1s 2ms/step
[11.892435378918387, 0.1858789622783661]
Overall accuracy = 0.1822485223765204


# German, Italian and Urdu Dataset Testing

## Italian,German and Urdu dataset Training and Testing

In [ ]:
# x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/EMOVO_URDU_EMODB/8classes_1D_CNN_65%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc2*b1 + acc3*c1)/(b1+c1))

Training
5065/5065 [==============================] - 8s 2ms/step
[0.015937080990310725, 0.9944718480110168]
Validation
1202/1202 [==============================] - 2s 2ms/step
[2.643213455371571, 0.6572379469871521]
Testing
1618/1618 [==============================] - 3s 2ms/step
[2.4399181983084848, 0.6545117497444153]
Overall accuracy = 0.6556737671507166


## Italian,German and Urdu Dataset Training and German Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/EMOVO_URDU_EMODB/8classes_1D_CNN_65%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',( acc2*b1 + acc3*c1)/(b1+c1))

Training
1733/1733 [==============================] - 3s 2ms/step
[0.002403312493722931, 0.9994229674339294]
Validation
428/428 [==============================] - 1s 2ms/step
[1.9716128634515209, 0.7242990732192993]
Testing
549/549 [==============================] - 1s 2ms/step
[2.751533870488568, 0.6502732038497925]
Overall accuracy = 0.682702141506035


## Italian,German and Urdu Dataset Training and Urdu Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdu_tr,ydu_tr,xdu_t,ydu_t,xdu_val,ydu_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/EMOVO_URDU_EMODB/8classes_1D_CNN_65%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',( acc2*b1 + acc3*c1)/(b1+c1))

Training
1150/1150 [==============================] - 2s 2ms/step
[0.0033155426393141566, 0.9991304278373718]
Validation
270/270 [==============================] - 1s 2ms/step
[1.579266733593411, 0.7851851582527161]
Testing
375/375 [==============================] - 1s 2ms/step
[1.1094363713264466, 0.8293333053588867]
Overall accuracy = 0.8108526856400246


## Italian,German and Urdu Dataset Training and Emovo Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdi_tr,ydi_tr,xdi_t,ydi_t,xdi_val,ydi_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/EMOVO_URDU_EMODB/8classes_1D_CNN_65%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc2*b1 + acc3*c1)/(b1+c1))

Training
2182/2182 [==============================] - 3s 2ms/step
[0.03333799102224963, 0.9880843162536621]
Validation
504/504 [==============================] - 1s 2ms/step
[3.783511990592593, 0.5317460298538208]
Testing
694/694 [==============================] - 1s 2ms/step
[2.9123296813250277, 0.5634005665779114]
Overall accuracy = 0.5500834659861403


# Italian Dataset Testing

## Emovo(Italian) dataset Training and Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdi_tr,ydi_tr,xdi_t,ydi_t,xdi_val,ydi_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/EMOVO/preTrained_MIX_8classes_1D_CNN_55%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc2*b1 + acc3*c1)/(b1+c1))

Training
2182/2182 [==============================] - 3s 2ms/step
[0.018905006071066485, 0.994958758354187]
Validation
504/504 [==============================] - 1s 2ms/step
[3.4431617032913935, 0.557539701461792]
Testing
694/694 [==============================] - 1s 2ms/step
[2.8414098233929286, 0.5706051588058472]
Overall accuracy = 0.5651085056327221


## EMOVO(Italian) Dataset Training and Urdu Dataset *Testing*

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdu_tr,ydu_tr,xdu_t,ydu_t,xdu_val,ydu_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/EMOVO/preTrained_MIX_8classes_1D_CNN_55%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc1*a1 + acc2*b1 + acc3*c1)/(total))

Training
1150/1150 [==============================] - 2s 2ms/step
[0.5749144756275675, 0.8860869407653809]
Validation
270/270 [==============================] - 1s 2ms/step
[2.6736559867858887, 0.6888889074325562]
Testing
375/375 [==============================] - 1s 2ms/step
[2.557734718322754, 0.6986666917800903]
Overall accuracy = 0.8172701929273047


## EMOVO(Italian) Dataset Training and Emodb Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/EMOVO/preTrained_MIX_8classes_1D_CNN_55%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc1*a1 + acc2*b1 + acc3*c1)/(total))

Training
1733/1733 [==============================] - 3s 2ms/step
[0.21030963509329054, 0.9417195320129395]
Validation
428/428 [==============================] - 1s 2ms/step
[2.628302157482254, 0.6355140209197998]
Testing
549/549 [==============================] - 1s 2ms/step
[3.3902886421954044, 0.5883424282073975]
Overall accuracy = 0.821771196685594


# Urdu Dataset Testing

## Urdu dataset Training and Testing

In [ ]:
# model = load_model('best_model_1D_CNN.h5')
# x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/Urdu_Dataset/Urdu_8classes_1D_CNN_85%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc2*b1 + acc3*c1)/(b1+c1))

Training
1159/1159 [==============================] - 2s 2ms/step
[0.0989914628145919, 0.9680759310722351]
Validation
270/270 [==============================] - 1s 2ms/step
[0.66096782622238, 0.8518518805503845]
Testing
366/366 [==============================] - 1s 2ms/step
[0.42507086535457705, 0.8579235076904297]
Overall accuracy = 0.8553459301309766


## Urdu Dataset Training and Berlin Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdg_tr,ydg_tr,xdg_t,ydg_t,xdg_val,ydg_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/Urdu_Dataset/Urdu_8classes_1D_CNN_85%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc1*a1 + acc2*b1 + acc3*c1)/(total))

Training
1748/1748 [==============================] - 3s 2ms/step
[8.503640713899031, 0.21567505598068237]
Validation
426/426 [==============================] - 1s 2ms/step
[8.160300946571457, 0.23239436745643616]
Testing
536/536 [==============================] - 1s 2ms/step
[8.099775278746192, 0.222014918923378]
Overall accuracy = 0.21955719370243734


## Urdu Dataset Training and EMOVO Dataset Testing

In [ ]:
# xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdi_tr,ydi_tr,xdi_t,ydi_t,xdi_val,ydi_val
# x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/Urdu_Dataset/Urdu_8classes_1D_CNN_85%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc1*a1 + acc2*b1 + acc3*c1)/(total))

Training
2206/2206 [==============================] - 3s 2ms/step
[10.079037068869347, 0.15548503398895264]
Validation
512/512 [==============================] - 1s 2ms/step
[9.78446738421917, 0.193359375]
Testing
662/662 [==============================] - 1s 2ms/step
[9.973932488087081, 0.16314199566841125]
Overall accuracy = 0.16272188938228335


# German Dataset Testing

## German dataset Training and Testing

In [ ]:
# x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/Emodb/Emodb_8classes_1D_CNN_68%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc2*b1 + acc3*c1)/(b1+c1))

Training
1748/1748 [==============================] - 3s 2ms/step
[0.06346781342982973, 0.9788329601287842]
Validation
426/426 [==============================] - 1s 2ms/step
[1.851806019393491, 0.6737089157104492]
Testing
536/536 [==============================] - 1s 2ms/step
[1.593299764305798, 0.7052238583564758]
Overall accuracy = 0.6912681768936824


## German Dataset Training and Urdu Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdu_tr,ydu_tr,xdu_t,ydu_t,xdu_val,ydu_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/Emodb/Emodb_8classes_1D_CNN_68%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc1*a1 + acc2*b1 + acc3*c1)/(total))

Training
1159/1159 [==============================] - 2s 2ms/step
[11.857266340510819, 0.22864538431167603]
Validation
270/270 [==============================] - 1s 2ms/step
[12.775508138868544, 0.22592592239379883]
Testing
366/366 [==============================] - 1s 2ms/step
[11.653262766902564, 0.24590164422988892]
Overall accuracy = 0.2317548753491351


## German Dataset Training and EMOVO Dataset Testing

In [ ]:
xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val = xdi_tr,ydi_tr,xdi_t,ydi_t,xdi_val,ydi_val
x_tr,y_tr,x_t,y_t,x_val,y_val = loadData1d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

def test(model,x_t,y_t):
	saved_model = model = load_model('/content/drive//My Drive/Emotion Detection/Models/Emodb/Emodb_8classes_1D_CNN_68%_1s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
acc1 = score[1]
print("Validation")
score = test(model,x_val,y_val)
acc2 = score[1]
print("Testing")
score = test(model,x_t,y_t)
acc3 = score[1]

a1 = x_tr.shape[0]
b1 = x_val.shape[0]
c1 = x_t.shape[0]
total = a1+b1+c1
print('Overall accuracy =',(acc1*a1 + acc2*b1 + acc3*c1)/(total))

Training
2206/2206 [==============================] - 3s 2ms/step
[8.941031503115367, 0.15820489823818207]
Validation
512/512 [==============================] - 1s 2ms/step
[9.822020545601845, 0.154296875]
Testing
662/662 [==============================] - 1s 2ms/step
[9.349943878427373, 0.17673715949058533]
Overall accuracy = 0.1612426050580465


# 2D CNN and LSTM

In [ ]:
# 2D cnn for SER

pw = 2 # Pool Layer Window size

def emo2d(input_shape, num_classes,args):
	
	model = Sequential(name='Emo2D')
	print("Hey",input_shape)
	# LFLB1
	model.add(Conv2D(filters = 64,kernel_size = 3,strides=1,padding='same',data_format='channels_last',input_shape=input_shape, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
	model.add(BatchNormalization())
	model.add(Activation('elu'))
	model.add(MaxPooling2D(pool_size = (int(2*pw/2),int(2*pw/2)), strides = int(pw/2),data_format='channels_last'))

	#LFLB2
	model.add(Conv2D(filters=64, kernel_size = 3, strides=1,padding='same', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
	model.add(BatchNormalization())
	model.add(Activation('elu'))
	model.add(MaxPooling2D(pool_size = (pw,pw), strides = pw,data_format='channels_last'))

	#LFLB3
	model.add(Conv2D(filters=128, kernel_size = 3, strides=1,padding='same', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
	model.add(BatchNormalization())
	model.add(Activation('elu'))
	model.add(MaxPooling2D(pool_size = (pw,pw), strides = pw,data_format='channels_last',))

	# #LFLB4
	model.add(Conv2D(filters=128, kernel_size = 3, strides=1,padding='same', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
	model.add(BatchNormalization())
	model.add(Activation('elu'))
	model.add(MaxPooling2D(pool_size = (pw,pw), strides = pw,data_format='channels_last'))

	# model.add(Flatten())
	model.add(Reshape((-1,1)))

	#LSTM
	model.add(LSTM(units=args.num_fc,return_sequences=False, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
	#FC
	model.add(Dense(units=100,activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))

	model.add(Dense(units=num_classes,activation='softmax'))

	#Model compilation	
	opt = optimizers.SGD(lr = args.learning_rate, decay=args.decay, momentum=args.momentum, nesterov=True)
	model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
	
	return model

def train(model,x_tr,y_tr,x_val,y_val,args):
	
	es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
	mc = ModelCheckpoint('best_model_2D_CNN_4s.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
	history = model.fit(x_tr,y_tr,epochs=args.num_epochs,batch_size=args.batch_size,validation_data=(x_val,y_val),callbacks=[es, mc])
	return model

def test(model,x_t,y_t):

	saved_model = load_model('best_model_2D_CNN_4s.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

def loadData2d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val):

	x_tr = []
	x_val = []
	x_t = []

	for a in xd_tr:
		S = librosa.feature.melspectrogram(y=a, sr=sr)
		x_tr.append(librosa.power_to_db(S, ref=np.max))
	
	for a in xd_val:
		S = librosa.feature.melspectrogram(y=a, sr=sr)
		x_val.append(librosa.power_to_db(S, ref=np.max))
	for a in xd_t:
		S = librosa.feature.melspectrogram(y=a, sr=sr)
		x_t.append(librosa.power_to_db(S, ref=np.max))

	x_tr,x_t,x_val = np.array(x_tr),np.array(x_t),np.array(x_val)

	x_tr = x_tr.reshape(-1,x_tr.shape[1],x_tr.shape[2],1)
	x_t = x_t.reshape(-1,x_t.shape[1],x_t.shape[2],1)
	x_val = x_val.reshape(-1,x_val.shape[1],x_val.shape[2],1)
	y_tr = to_categorical(yd_tr)
	y_t = to_categorical(yd_t)
	y_val = to_categorical(yd_val)
	return np.array(x_tr),y_tr,np.array(x_t),y_t,np.array(x_val),y_val

class arguments(object):        
		def __init__(self):
				self.num_fc = 256
				self.batch_size = 32
				self.num_epochs = 20 #best model will be saved before number of epochs reach this value
				self.learning_rate = 0.005
				self.decay = 1e-6
				self.momentum = 0.9

#load data	
# x_tr,y_tr,x_t,y_t,x_val,y_val = loadData2d(xd_tr,yd_tr,xd_t,yd_t,xd_val,yd_val)	

print(x_tr.shape)
print(x_val.shape)
print(x_t.shape)

args = arguments()

#define model
model = emo2d(input_shape=x_tr.shape[1:],num_classes=len(np.unique(np.argmax(y_tr, 1))),args=args)
model.summary()

#train model
model = train(model,x_tr,y_tr,x_val,y_val,args=args)

#test model
score = test(model,x_t,y_t)


In [ ]:
0# !gsutil -q -m cp "best_model_2D_CNN_8s_67%.h5" "/content/drive//My Drive/Emotion Detection/Models/Emodb"

In [ ]:
# model = load_model('/content/drive//My Drive/Emotion Detection/best_model_2D_CNN_8s_67%.h5')

def test(model,x_t,y_t):

	saved_model = load_model('/content/drive//My Drive/Emotion Detection/Models/Emodb/best_model_2D_CNN_8s_67%.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=128)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
print("Validation")
score = test(model,x_val,y_val)
print("Testing")
score = test(model,x_t,y_t)


Training
345/345 [==============================] - 1s 3ms/step
[0.9619181715923807, 0.8231884241104126]
Validation
85/85 [==============================] - 0s 5ms/step
[1.2091891765594482, 0.7882353067398071]
Testing
106/106 [==============================] - 0s 4ms/step
[0.9688431620597839, 0.7547169923782349]


In [ ]:
def test(model,x_t,y_t):

	saved_model = model = load_model('best_model_2D_CNN.h5')
	score = saved_model.evaluate(x_t,y_t,batch_size=120)
	print(score)
	return score

#test model
print("Training")
score = test(model,x_tr,y_tr)
print("Validation")
score = test(model,x_val,y_val)
print("Testing")
score = test(model,x_t,y_t)


Training
345/345 [==============================] - 1s 2ms/step
[1.1327433378800102, 0.7971014380455017]
Validation
85/85 [==============================] - 0s 3ms/step
[1.7833393812179565, 0.5764706134796143]
Testing
106/106 [==============================] - 0s 3ms/step
[1.8798648118972778, 0.5283018946647644]


In [ ]:
model = load_model('best_model_2D_CNN.h5')

predict = np.argmax(np.array(model.predict(x_tr)),axis = 1)
print(predict.reshape((-1,1)).shape)
# print(predict)
abc = np.array(yd_tr,dtype = np.dtype(np.int8))
# print(abc)
print(abc.reshape((-1,1)).shape)
print(confusion_matrix(abc,predict))

score = model.evaluate(x_tr,y_tr,batch_size=120)
print(score)

# print("Total Data {'W': 127, 'L': 81, 'E': 46, 'A': 69, 'F': 71, 'T': 62, 'N': 79}")
# print("Class: W train: 82 val: 20 test: 25")
# print("Class: L train: 52 val: 13 test: 16")
# print("Class: E train: 30 val: 7 test: 9")
# print("Class: A train: 44 val: 11 test: 14")
# print("Class: F train: 46 val: 11 test: 14")
# print("Class: T train: 40 val: 10 test: 12")
# print("Class: N train: 50 val: 13 test: 16")


(345, 1)
(345, 1)
[[ 0  0  1  7 74  0  0]
 [ 0 51  0  0  0  1  0]
 [ 0  0 25  2  0  0  3]
 [ 0  0  0 28  0  0 16]
 [ 0  0  0 46  0  0  0]
 [ 0 15  0  0  0 26  0]
 [ 0 15  0  0  0  0 35]]
345/345 [==============================] - 1s 2ms/step
[2.5316155371458633, 0.47826087474823]
